In [170]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [171]:
#df = final_df_sentiment
final_df_sentiment = pd.read_json("final_df_sentiment.json")
final_df_knn = pd.read_json('final_df_knn.json')

In [172]:
rating_review = final_df_sentiment[['reviewText','overall']]

In [173]:
rating_review.head()

,reviewText,overall
0,"Not much to write about here, but it does exac...",5
1,The product does exactly as it should and is q...,5
10,"Monster makes a wide array of cables, includin...",5
100,The listing for this item can be confusing if ...,5
1000,The tips of my fingers often get pretty dry an...,5


In [174]:
rating_review["opinion"] = rating_review["overall"].apply(lambda value: "good" if value >= 3 else "bad")

c:\users\balachandar\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [175]:
rating_review[rating_review.overall == 2].head(3)

,reviewText,overall,opinion
10027,Soooo bummed. I simply love the sound on these...,2,bad
10045,"I love Tascam gear, and have used it for nearl...",2,bad
10110,"The concept of this guitar stand is great, and...",2,bad


In [176]:
rating_review["reviewText"].head(3)

0     Not much to write about here, but it does exac...
1     The product does exactly as it should and is q...
10    Monster makes a wide array of cables, includin...
Name: reviewText, dtype: object

In [177]:
regular_expression = re.compile('[^a-z]+')
def process_text(review_text):
    if type(review_text) is str:
        review_text = review_text.lower()
        review_text = regular_expression.sub(' ', review_text).strip()
    return review_text

rating_review["processedReview"] = rating_review["reviewText"].apply(process_text)

c:\users\balachandar\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [178]:
rating_review.head(3)

,reviewText,overall,opinion,processedReview
0,"Not much to write about here, but it does exac...",5,good,not much to write about here but it does exact...
1,The product does exactly as it should and is q...,5,good,the product does exactly as it should and is q...
10,"Monster makes a wide array of cables, includin...",5,good,monster makes a wide array of cables including...


In [179]:
rating_review = rating_review.drop(['reviewText', 'overall'],1)
rating_review.head(5)

,opinion,processedReview
0,good,not much to write about here but it does exact...
1,good,the product does exactly as it should and is q...
10,good,monster makes a wide array of cables including...
100,good,the listing for this item can be confusing if ...
1000,good,the tips of my fingers often get pretty dry an...


In [180]:
train_data = rating_review
(len(train_data), len(final_df_knn))

(9718, 536)

In [181]:
train_data.head(3)

,opinion,processedReview
0,good,not much to write about here but it does exact...
1,good,the product does exactly as it should and is q...
10,good,monster makes a wide array of cables including...


In [182]:
#CountVectorizer to get bag of words model
count_vectorizer = CountVectorizer(min_df = 1, ngram_range = (1, 5))
X_train_bags = count_vectorizer.fit_transform(train_data["processedReview"])

#tfidf to bag of words
tfidf_transformer = TfidfTransformer()
X_train_data = tfidf_transformer.fit_transform(X_train_bags)

Y_train_data = train_data["opinion"]
Y_test_data = final_df_knn["opinion"]
predicted_output = dict();

KeyError: 'opinion'

In [140]:
#getting ready test data
X_test_counts = count_vectorizer.transform(test_data["processedReview"])
X_test_data = tfidf_transformer.transform(X_test_counts)

In [142]:
logistic_regres = LogisticRegression(C=1e5)
logistic_regres.fit(X_train_data, Y_train_data)
predicted_output['logistic'] = logistic_regres.predict(X_test_data)

In [143]:
predicted_output['logistic']

array(['good', 'good', 'good', ..., 'good', 'good', 'good'], dtype=object)

In [137]:
accuracy_score(Y_test_data, predicted_output['logistic'])

0.95936213991769548

In [161]:
X_test_data

1944

In [169]:
#testing
test_output = dict();
item_test_bags = count_vectorizer.transform([item_test_data])
item_test_processed_data = tfidf_transformer.transform(item_test_bags)
test_output['logistic'] = logistic_regres.predict_proba(final_df_knn[])
predicted_rating = test_output['logistic'][:,1]*5


In [129]:
def getPredictedRatings(model, item_test_data):
    item_test_bags = count_vectorizer.transform([item_test_data])
    item_test_processed_data = tfidf_transformer.transform(item_test_bags)
    result = model.predict(item_test_processed_data)
    print(result)
    probability = model.predict_proba(item_test_processed_data)[0]
    print(probability)
#     print("Sample estimated as %s: negative prob %f, positive prob %f" % (result.upper(), , ))
#     return (probability[0]*5,probability[1]*5)

getPredictedRatings(logistic_regres, "The product does exactly as it should and is")
getPredictedRatings(logistic_regres, "The concept of this guitar stand is great,")


['good']
[  2.57847793e-05   9.99974215e-01]
['good']
[  1.29138757e-04   9.99870861e-01]
